### Engineers' Salary Prediction Challenge

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#### Wczytanie danych

In [ ]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

train.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
def compare_train_test(column):
    train_values = train[column].value_counts().reset_index(name='train_count')
    test_values = test[column].value_counts().reset_index(name='test_count')
    return pd.merge(test_values, train_values, how='outer', on=column)

c_job_title = compare_train_test('job_title')
c_job_state = compare_train_test('job_state')

### <center>Braki w kolumnach</center>

In [ ]:
train.isnull().sum().sort_values(ascending=False)

In [ ]:
test.isnull().sum().sort_values(ascending=False)

### <center>Analiza kolumn job_desc</center>

In [ ]:
job_desc_columns = [col for col in train.columns if col.startswith('job_desc_')]

#### Które kolumny job_desc wpływają na daną klasę

In [ ]:
f_vals, p_vals = f_classif(train[job_desc_columns], train['salary_category'])
anova_f = pd.DataFrame(data={'column': job_desc_columns, 'f_val': f_vals, 'p_val': p_vals})

anova_f

In [ ]:
important_columns = anova_f.loc[(anova_f['f_val'] >= 10) & (anova_f['p_val'] <= 0.05), 'column'].tolist()

len(important_columns)

#### Zostawiamy tylko te najważniejsze z punktów widzenia rozróżnienia klas

In [ ]:
important_train = train[important_columns]
important_test = test[important_columns]

train.drop(columns=job_desc_columns, inplace=True)
test.drop(columns=job_desc_columns, inplace=True)

train = pd.concat([train, pd.DataFrame(data=important_train, columns=important_columns)], axis=1)
test = pd.concat([test, pd.DataFrame(data=important_test, columns=important_columns)], axis=1)

job_desc_columns = important_columns

#### PCA dla kolumn job_desc

In [ ]:
pca_n_components = 5

standard_scaler = StandardScaler()
train[job_desc_columns] = standard_scaler.fit_transform(train[job_desc_columns])
test[job_desc_columns] = standard_scaler.transform(test[job_desc_columns])

job_desc_pca = PCA(
    n_components=pca_n_components,
    random_state=42,
    svd_solver='auto',
    whiten=False
).fit(train[job_desc_columns])

train_pca_decomposed = job_desc_pca.transform(train[job_desc_columns])
test_pca_decomposed = job_desc_pca.transform(test[job_desc_columns])

train.drop(columns=job_desc_columns, inplace=True)
test.drop(columns=job_desc_columns, inplace=True)

pca_columns = [f'pca_{i}' for i in range(job_desc_pca.n_components_)]

train = pd.concat([train, pd.DataFrame(data=train_pca_decomposed, columns=pca_columns)], axis=1)
test = pd.concat([test, pd.DataFrame(data=test_pca_decomposed, columns=pca_columns)], axis=1)

#### Ilość zachowanych informacji przez każdą składową główną PCA

In [ ]:
preserved_info = pd.DataFrame(data={
    'components': pca_columns,
    'variance': job_desc_pca.explained_variance_ratio_ * 100
})

plt.figure(figsize=(12, 7))
sns.barplot(
    data=preserved_info,
    x='components',
    y='variance',
    color='skyblue',
    edgecolor='black'
)

plt.xlabel('Składowe główne', fontsize=14)
plt.ylabel('Procent zachowanych informacji [%]', fontsize=14)
plt.title('Analiza PCA: Procent wariancji wyjaśnionej przez składowe główne', fontsize=16, pad=20)

for i, v in enumerate(preserved_info['variance']):
    plt.text(i, v + 0.5, f'{v:.1f}%', ha='center', fontsize=12)

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

#### PCA, a klasa

In [ ]:
sns.pairplot(
    data=train.loc[:, pca_columns + ['salary_category']],
    hue='salary_category'
)

In [ ]:
sns.pairplot(
    data=train.loc[:, pca_columns],
)

### <center>Miesiąc i rok</center>

In [ ]:
(train['job_posted_date'].isna().sum(), test['job_posted_date'].isna().sum())

In [ ]:
train['job_posted_date'] = train['job_posted_date'].fillna(train['job_posted_date'].mode()[0])

train['job_posted_date'] = pd.to_datetime(train['job_posted_date'], format='%Y/%m')
train['month'] = train['job_posted_date'].dt.month
train['year'] = train['job_posted_date'].dt.year
train.drop(columns=['job_posted_date'], inplace=True)

test['job_posted_date'] = pd.to_datetime(test['job_posted_date'], format='%Y/%m')
test['month'] = test['job_posted_date'].dt.month
test['year'] = test['job_posted_date'].dt.year
test.drop(columns=['job_posted_date'], inplace=True)

### <center>Porównanie wybranych kolumn kategorycznych w train/test</center>

#### <center>Stan w którym jest oferta pracy "job_state"</center>

In [ ]:
c_job_state

In [ ]:
(train['job_state'].isna().sum(), test['job_state'].isna().sum())

#### <center>WYKRES: Region USA, a wysokość pensji</center>

In [ ]:
def plot_bar(column, width, xlabel, ylabel, title, main_ax=None, fontsize=16, show_legend=True, rotation=90):
    salary_types = train['salary_category'].unique()
    show_every_column = train.groupby([column, 'salary_category']).size().unstack(fill_value=0).reindex(columns=salary_types, fill_value=0).stack().reset_index(name='Count')

    x_axis = np.arange(len(show_every_column[column].unique()))
    my_xticks = [x_axis - width, x_axis, x_axis + width]
    colors = ['gold', 'silver', 'brown']

    if main_ax is None:
        f, a = plt.subplots(figsize=(10, 8))
    else:
        a = main_ax

    for i, salary in enumerate(salary_types):
        a.bar(
            x=my_xticks[i],
            height=show_every_column.loc[(show_every_column['salary_category'] == salary), 'Count'],
            width=width,
            label=salary,
            color=colors[i],
            edgecolor='black',
            alpha=1
        )

    a.grid(True)
    a.set_xticks(x_axis)
    a.set_xticklabels(show_every_column[column].unique(), rotation=rotation)
    a.set_xlabel(xlabel, fontsize=fontsize)
    a.set_ylabel(ylabel, fontsize=fontsize)
    a.set_title(title, fontsize=fontsize)
    if show_legend:
        a.legend(title='Kategoria wynagrodzenia')

plot_bar(
    column='job_state',
    width=0.2,
    xlabel='Stan',
    ylabel='Liczba ofert',
    title='Kategoria zarobkowa w stanach'
)

#### Zmiana stanów na regiony USA

In [ ]:
west_states = [
    'AK', 'AZ', 'CA', 'CO', 'HI', 'ID',
    'MT', 'NV', 'NM', 'OR', 'UT', 'WA', 'WY'
]

central_states = [
    'AR', 'IA', 'IL', 'IN', 'KS', 'KY', 'LA',
    'MI', 'MN', 'MO', 'ND', 'NE', 'OH',
    'OK', 'SD', 'TX', 'WI'
]

east_states = [
    'AL', 'CT', 'DE', 'FL', 'GA', 'MA', 'MD',
    'ME', 'MS', 'NC', 'NH', 'NJ', 'NY', 'PA',
    'RI', 'SC', 'TN', 'VA', 'VT', 'WV', 'DC'
]

def map_state_to_region(state):
    if state in west_states:
        return 'West'
    elif state in central_states:
        return 'Central'
    else:
        return 'East'

train['job_state'] = train['job_state'].fillna(train['job_state'].mode()[0])
test['job_state'] = test['job_state'].fillna(test['job_state'].mode()[0])
train['job_region'] = train['job_state'].apply(map_state_to_region)
test['job_region'] = test['job_state'].apply(map_state_to_region)

In [ ]:
pd.crosstab(train['salary_category'], train['job_region'], normalize='index')

#### <center>WYKRES: Region USA, a wysokość pensji</center>

In [ ]:
plot_bar(
    column='job_region',
    width=0.2,
    xlabel='Region',
    ylabel='Liczba ofert',
    title='Kategoria zarobkowa w regionach'
)

#### Co zrobić z job_state?

Testowano:
- usunięcie -> tak sobie
- zmiana na "job_region" i kodowanie one-hot -> narazie najlepiej
- zmiana na "job_region" i kodowanie labelencoder -> przetestować

In [ ]:
job_region_one_hot = OneHotEncoder(sparse_output=False, dtype=int)
job_region_one_hot.fit(train[['job_region']])

train = pd.concat([train, pd.DataFrame(job_region_one_hot.transform(train[['job_region']]), columns=job_region_one_hot.categories_[0])], axis=1)
test = pd.concat([test, pd.DataFrame(job_region_one_hot.transform(test[['job_region']]), columns=job_region_one_hot.categories_[0])], axis=1)

train.drop(columns=['job_state', 'job_region'], inplace=True)
test.drop(columns=['job_state', 'job_region'], inplace=True)

#### <center>Stanowisko pracy "job_title"</center>

In [ ]:
c_job_title

In [ ]:
(train['job_title'].isna().sum(), test['job_title'].isna().sum())

#### <center>WYKRES: Stanowisko pracy a wysokość pensji</center>

In [ ]:
plot_bar(
    column='job_title',
    width=0.2,
    xlabel='Stanowisko',
    ylabel='Liczba ofert',
    title='Kategorie zarobkowe dla każdej oferty pracy'
)

#### <center>Kolumny "feature_"</center>

In [ ]:
feature_columns = [col for col in train.columns if col.startswith('feature_')]

train[feature_columns].info()

#### <center>WYKRES: Podział wartości kolumn feature według wysokość pensji</center>

In [ ]:
exclude_float = train[feature_columns].select_dtypes(exclude=np.float64).columns
fig, ax = plt.subplots(nrows=5, ncols=2, figsize=(12, 20))

for i, col in enumerate(exclude_float):
    x, y = divmod(i, 2)
    plot_bar(
        column=col,
        width=0.2,
        xlabel=col,
        ylabel='Liczba ofert',
        title='',
        main_ax=ax[x, y],
        fontsize=12,
        show_legend=True,
        rotation=0
    )

#### feature_1

In [ ]:
grouped_feature_1 = train.groupby(['salary_category', 'feature_1']).size().unstack(fill_value=0).T

grouped_feature_1

In [ ]:
test['feature_1'].value_counts()

#### Korelacja kolumn feature

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    train[feature_columns].corr(numeric_only=True),
    annot=True,
    fmt='.2f',
    cmap='viridis',
    linewidths=0.5,
    linecolor='black',
    square=True,
    cbar=True
)

#### feature_2 (bo jest najwyżej w importance dla xgb, lgb i rf)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(
    data=train,
    x='feature_2',
    hue='salary_category',
    kde=True,
    palette='Set2',
    multiple='stack',
    bins=100,
)

plt.title('Rozkład feature_2 według kategorii wynagrodzenia w zbiorze treningowym', fontsize=14)
plt.xlabel('feature_2', fontsize=12)
plt.ylabel('Liczba przypadków', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(
    data=test,
    x='feature_2',
    kde=True,
    bins=100,
)

plt.title('Rozkład feature_2 w zbiorze testowym', fontsize=14)
plt.xlabel('feature_2', fontsize=12)
plt.ylabel('Liczba przypadków', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

#### feature_10 (bo jest najwięcej NaN)

W feature_10 okres 06-08/2024 ma najwięcej brakujących wartości dla zbioru treningowego.

Tak samo jest w zbiorze testowym.

In [ ]:
train_grouped_feature_10 = train.loc[train['feature_10'].isna(), ['month', 'year']]

train_grouped_feature_10.groupby(['year', 'month']).size().sort_values(ascending=False).head(n=3)

In [ ]:
test_grouped_feature_10 = test.loc[test['feature_10'].isna(), ['month', 'year']]

test_grouped_feature_10.groupby(['year', 'month']).size().sort_values(ascending=False).head(n=3)

#### Kodowanie LabelEncoder na salary_category

In [ ]:
salary_category = train['salary_category']
salary_category_encoder = LabelEncoder().fit(salary_category)
train['salary_category'] = salary_category_encoder.transform(train['salary_category'])

#### Korelacja dla całego zbioru

In [ ]:
plt.figure(figsize=(20, 12))
sns.heatmap(
    train.corr(numeric_only=True),
    annot=True,
    fmt='.2f',
    cmap='viridis',
    linewidths=0.5,
    linecolor='black',
    square=True,
    cbar=True
)

#### Wartości korelacji dla kolumny salary_category

Usunięcie kolumn mało informatywnych i kolumn wysoko skorelowanych.

In [ ]:
test_obs = test['obs']
train.drop(columns=['obs'], inplace=True)
test.drop(columns=['obs'], inplace=True)

corr = train.corr(numeric_only=True)[['salary_category']]
columns_to_drop = corr.loc[(corr['salary_category'] < 0.02) & (corr['salary_category'] > -0.02), :].index.values

train.drop(columns=columns_to_drop, inplace=True)
test.drop(columns=columns_to_drop, inplace=True)

columns_to_drop

In [ ]:
def drop_high_correlated_columns():
    matrix = train.corr(numeric_only=True).abs()
    upper_t = matrix.where(np.triu(np.ones_like(matrix, dtype=np.bool_), k=1))
    return [col for col in upper_t.columns if any(upper_t[col] > 0.9)]

high_correlated_columns = drop_high_correlated_columns()
train.drop(columns=high_correlated_columns, inplace=True)
test.drop(columns=high_correlated_columns, inplace=True)

high_correlated_columns

#### Przygotowanie danych do ostatecznego zapisu

In [ ]:
train['salary_category'] = salary_category_encoder.inverse_transform(train['salary_category'])
test = pd.concat([test_obs, test], axis=1)

In [ ]:
train.head()

In [ ]:
test.head()

#### Sprawdzenie PSI dla poszczególnych kolumn w train i test

PSI (Population Stability Index) - jest to metoda używana do ilościowego określania zmian rozkładów jednej zmiennej (jednowymiarowej) między dwoma zbiorami danych.

Wartość PSI:
- PSI < 0.1 -> nie ma żadnych znaczących różnic między tą samą zmienną w dwóch zbiorach
- 0.1 <= PSI < 0.25 -> cecha posiada umiarkowane różnice w rozkładzie dla dwóch zbiorów
- 0.25 > PSI -> sugeruje znaczące różnice w rozkładzie cechy dla dwóch zbiorów

In [ ]:
train_psi = pd.read_csv(os.path.join('data', 'train.csv'))
test_psi = pd.read_csv(os.path.join('data', 'test.csv'))

train_psi['job_posted_date'] = train_psi['job_posted_date'].fillna(train_psi['job_posted_date'].mode()[0])
train_psi['job_state'] = train_psi['job_state'].fillna(train_psi['job_state'].mode()[0])
test_psi['job_state'] = test_psi['job_state'].fillna(test_psi['job_state'].mode()[0])
train_psi['feature_10'] = train_psi['feature_10'].fillna(0)
test_psi['feature_10'] = test_psi['feature_10'].fillna(0)

train_psi['job_posted_date'] = pd.to_datetime(train_psi['job_posted_date'], format='%Y/%m')
train_psi['month'] = train_psi['job_posted_date'].dt.month.astype('category')
train_psi['year'] = train_psi['job_posted_date'].dt.year.astype('category')
train_psi.drop(columns=['job_posted_date'], inplace=True)

test_psi['job_posted_date'] = pd.to_datetime(test_psi['job_posted_date'], format='%Y/%m')
test_psi['month'] = test_psi['job_posted_date'].dt.month.astype('category')
test_psi['year'] = test_psi['job_posted_date'].dt.year.astype('category')
test_psi.drop(columns=['job_posted_date'], inplace=True)

train_psi.drop(columns=['obs', 'salary_category'], inplace=True)
test_psi.drop(columns=['obs'], inplace=True)

categories = train_psi.select_dtypes(include=['object', 'bool']).columns

train_psi[categories] = train_psi[categories].astype('category')
test_psi[categories] = test_psi[categories].astype('category')

In [ ]:
def psi_numerical(reference, monitored):
    concatenated = np.concatenate((reference, monitored))

    _, bin_edges = np.histogram(concatenated, bins='doane')

    reference_hist, _ = np.histogram(reference, bins=bin_edges)
    monitored_hist, _ = np.histogram(monitored, bins=bin_edges)

    reference_proportions = reference_hist / np.sum(reference_hist)
    monitored_proportions = monitored_hist / np.sum(monitored_hist)

    monitored_proportions = np.where(monitored_proportions == 0, 1e-6, monitored_proportions)
    reference_proportions = np.where(reference_proportions == 0, 1e-6, reference_proportions)

    psi_values = (monitored_proportions - reference_proportions) * np.log(monitored_proportions / reference_proportions)
    psi = np.sum(psi_values)

    return psi

psi_numerical_values = [psi_numerical(train_psi[column], test_psi[column]) for column in train_psi.select_dtypes(exclude=['category']).columns]
pd.DataFrame({'column': train_psi.select_dtypes(exclude=['category']).columns, 'psi': psi_numerical_values}).sort_values(by='psi', ascending=False).head(10)

In [ ]:
def psi_categorical(reference, monitored):
    rc = reference.value_counts(normalize=True)
    mc = monitored.value_counts(normalize=True)

    all_categories = set(rc.index).union(set(mc.index))
    rc = rc.reindex(all_categories, fill_value=1e-6)
    mc = mc.reindex(all_categories, fill_value=1e-6)

    psi_values = (mc - rc) * np.log(mc / rc)
    return np.sum(psi_values)

psi_categorical_values = [psi_categorical(train_psi[column], test_psi[column]) for column in train_psi.select_dtypes(include=['category']).columns]
pd.DataFrame({'column': train_psi.select_dtypes(include=['category']).columns, 'psi': psi_categorical_values}).sort_values(by='psi', ascending=False).head(10)

In [ ]:
train.info()

In [ ]:
test.info()

#### Zapisz przygotowane dane

In [ ]:
if 'prepared' not in os.listdir(os.getcwd()):
    os.mkdir('prepared')

train.to_csv(os.path.join('prepared', 'train_cat.csv'), index=False)
test.to_csv(os.path.join('prepared', 'test_cat.csv'), index=False)